# Filtered state preparation with GQSP

Ground state preparation is a fundamental task in quantum computation with applications in chemistry, material sciences and physics. 
In this tutorial, we explore how to use [Generalized Quantum Signal Processing](https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.5.020368) for this task:
starting form an initial state, we apply a Gaussian filter to enhance the overlap of the prepared state with the ground state.

Let us consider an $L$-qubit antiferromagnetic Heisenberg model

$$ H = \sum_{i=0}^{L-1}(X_iX_{i+1} + Y_iY_{i+1} + Z_iZ_{i+1}) $$

As initial system state we use a tensor product of Singlet states

$$ \ket{\text{Singlet}} = 2^{-L/4}(\ket{10}-\ket{01})^{\otimes L/2} $$

of consecutive qubits.


In [ ]:
from qrisp import *
from qrisp.gqsp import GQSP
from qrisp.operators import X, Y, Z
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx


def generate_1D_chain_graph(L):
    graph = nx.Graph()
    graph.add_edges_from([(k, (k+1)%L) for k in range(L-1)]) 
    return graph


# Define Heisenberg Hamiltonian with spectrum in [-1,1].
L = 10
G = generate_1D_chain_graph(L)
H = (1 / (3 * L - 3)) * sum((X(i) * X(j) + Y(i) * Y(j) + Z(i) * Z(j)) \
                            for i, j in G.edges())
M = nx.maximal_matching(G)
U0 = create_heisenberg_init_function(M)


# Define initial state preparation function 
# preparing a tensor product of Singlet states.
def psi_prep():
    operand = QuantumVariable(H.find_minimal_qubit_amount())
    U0(operand)
    return operand


# Calculate the energy for the initial state.
E = H.expectation_value(psi_prep, precision=0.001)()
print(E)

Let us take a closer look at the spectrum of this Hamiltonian and the inital state $\ket{\psi_0}=\ket{\text{Singlet}}$. 
First, we compute all eigenvalues and the belonging eigenvectors. Secondly, we calculate the fidelities of the initial state $\ket{\psi_0}$ and each eigenstate $\ket{\lambda_k}$.
Next, we find the indices $k$ of the eigenstates $\ket{\lambda_k}$ that have a significant overlap with the initial state, i.e., $F_k=|\langle \lambda_k | \psi_0 \rangle|^2>0.01$.

In [ ]:
H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)

idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
eigvecs_sorted = eigvecs[:, idx]
print(f'Ground state energy: {eigvals_sorted[0]}\n')

psi0 = psi_prep().qs.statevector_array()
fidelities = np.abs(eigvecs_sorted.conj().T @ psi0) ** 2

threshold = 0.01
significant_mask = fidelities > threshold
indices = np.where(significant_mask)[0]

print(f'Eigenvalues (mask): {eigvals_sorted[indices]}\n')
print(f'Fidelities (mask): {fidelities[indices]}\n')

We observe that the initial state already has fidelity $F_0 \approx 0.68$ with the ground state $\ket{\lambda_0}$ with energy $\lambda_0 \approx -0.63$. 
Moreover, the inital state has a significant overlap with only 5 heigher energy eigenstates.

Starting at our initial state $\ket{\psi_0}$, we aim to prepare a state with an even higher overlap with the ground state $\ket{\psi_0}$. 
Therefore, we aim to apply a Gaussian filter centered at $\lambda_0$ which suppresses heigher energy components: indeed for 

$$ H = \sum_i\lambda_i\ket{\lambda_i}\bra{\lambda_i}\quad \text{and}\quad \ket{\psi_0} = \sum_i\alpha_i\ket{\lambda_i} $$

the state $\ket{\psi}$ after applying the filter $f$ is

$$ \ket{\psi} = \frac{f(H)\ket{\psi_0}}{\|f(H)\ket{\psi_0}\|} $$

where

$$ f(H)\ket{\psi_0} = \left(\sum_i f(\lambda_i)\ket{\lambda_i}\bra{\lambda_i}\right) \sum_j\alpha_j\ket{\lambda_j} = \sum_i f(\lambda_i)\alpha_i\ket{\lambda_i} $$

Thus, the magnitude of the amplitude of $\ket{\lambda_i}$ for $i>0$ is suppressed relative to the amplitude of $\ket{\lambda_0}$ in the state $\ket{\psi}$.
This filter is approximated by Chebyshev polynomials of the first kind.

In [ ]:
from numpy.polynomial import Chebyshev
from numpy.polynomial.chebyshev import chebval

# Define the Gaussian centered at minimal eigenvalue lambda_0 on [-1, 1].
mu = eigvals_sorted[0]
sigma = 0.05

def gaussian(x):
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

# Chebyshev fit
x_nodes = np.cos(np.pi * np.arange(201) / 200)
f_nodes = gaussian(x_nodes)
cheb_fit = Chebyshev.fit(x_nodes, f_nodes, deg=100)
cheb_coeffs = cheb_fit.coef

x_plot = np.linspace(-1, 1, 2000)
f_gaussian = gaussian(x_plot)
f_cheb_10 = chebval(x_plot, cheb_coeffs[:11]) 
f_cheb_20 = chebval(x_plot, cheb_coeffs[:21])  

# Plot
plt.figure(figsize=(10, 5))
plt.plot(x_plot, f_gaussian, color='#20306f', label='Gaussian', linewidth=2)
plt.plot(x_plot, f_cheb_10, color='#6929C4', label='d=10', linestyle='--')
plt.plot(x_plot, f_cheb_20, color='#444444', label='d=20', linestyle='--')
plt.axvline(mu, color='k', linestyle=':', alpha=0.4)
plt.xlabel('x', fontsize=15)
plt.xticks(fontsize=12)
plt.ylabel('f(x)', fontsize=15)
plt.yticks(fontsize=12)
plt.title('Chebyshev approximation of Gaussian filter', fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.tight_layout()
plt.show()

Let us now apply the filter using the built-in features of Qrisp's BlockEncoding programming abstraction.

In [ ]:
from qrisp.block_encodings import BlockEncoding

BE = BlockEncoding.from_operator(H)

# Apply degree 10 Chebyshev approximation of Gaussian filter.
BE_filter = BE.poly(cheb_coeffs[:11], kind="Chebyshev")

def filtered_psi_prep():
    operand = BE_filter.apply_rus(psi_prep)()
    return operand

In [ ]:
@jaspify(terminal_sampling=True)
def main(): 

    E = H.expectation_value(filtered_psi_prep, precision=0.001)()
    return E

#print(main())
# -0.6186466466466466

Indeed, we achieve a significantly lower energy -- hence, a higher overlap with the ground state -- for the transformed state after QSP. 

**But, what just happened?**

Actually, we use GQSP to apply an approximation to the Gaussian filter in Chebyshev basis.
While it is obvious to apply transformations in Fourier basis with GQSP, is may be used in this setting as well.
Let us recall the notion of a block-encoding:

Block-encoding is a foundational technique that enables the implementation of non-unitary operations on a quantum computer by embedding them into a larger unitary operator.
Consider a normalized Pauli operator $H=\sum_i\alpha_iP_i$ where $\alpha_i>0$ are real coefficients such that $\sum_i|\alpha_i|=1$, and $P_i$ are Pauli strings acting on $n$ qubits (including ther respective signs).
We embed $H$ into the upper-left block of a unitary matrix $U$:

$$ U = \begin{pmatrix} H & * \\ * & * \end{pmatrix} $$

More formally, a block-encoding of an operator $H$ (not necessarily unitary) acting on a Hilbert space $\mathcal H_{s}$ 
is a unitary acting on $\mathcal H_a\otimes H_s$ (for some auxiliary Hilbert space $\mathcal H_a$) such that

$$ (\bra{0}_a \otimes \mathbb I_s) U (\ket{0}_a \otimes \mathbb I_s) = H $$

Given a block-encoding unitary $U$ of a Hamiltonian $H$, such that $U^2=I$, let

$$ R=(2\ket{0}_a\bra{0}_a-I_a)\otimes I_s $$

be the reflection around $\ket{0}_a$. Then $(RU)^k$ is a block-encoding of $T_k(H)$ where $T_k$ is the $k$-th Chebyshev polynomial of the first kind (see [Kirby et al.](https://quantum-journal.org/papers/q-2023-05-23-1018/pdf/)).

That is, using $RU$ as our unitary for GQSP, we effectively apply a polynomial in Chebyshev basis.

First, we construct a block-encoding for the Pauli operator $H=\sum_i\alpha_iP_i$ using the Linear Combination of Unitaries (LCU) protocol.

We build our block-encoding unitary as $U=\text{PREP}\cdot \text{SELECT}\cdot \text{PREP}^{\dagger}$ where

$$ \text{SELECT} = \sum_i\ket{i}_a\bra{i}_a\otimes P_i $$

and

$$ \text{PREP}\ket{0} = \sum_i\sqrt{\alpha_i}\ket{i}_a $$

In [ ]:
# Returns a list of unitaries, i.e., functions performing in-place operations 
# on the operand quantum variable, and a list of coefficients.
unitaries, coeffs = H.unitaries()
n = len(unitaries).bit_length() # Number of qubits for auxiliary variable.

# Define block-encoding unitary using LCU = PREP SELECT PREP_dg.
def U(case, operand):
    with conjugate(prepare)(case, coeffs):
        q_switch(case, unitaries, operand) # quantum switch aka SELECT

Next, we define a function for performing the filtered state prepation using GQSP.

In [ ]:
@RUS
def filtered_psi_prep():

    # Qubitization step: RU^k is a block-encoding of T_k(H).
    def RU(case, operand):
        U(case, operand)
        reflection(case) # Reflection around |0>.


    qbl = QuantumBool()
    # Pauli block-encoding requires one auxiliary variable.
    case = QuantumFloat(n)
    operand = psi_prep()

    # Apply degree 10 Chebyshev approximation of Gaussian filter.
    GQSP(qbl, case, operand, unitary=RU, p=cheb_coeffs[:11], k=0)

    # Protocol is successful if all auxiliary variables are measured in state |0>.
    success_bool = (measure(qbl) == 0) & (measure(case) == 0)
    reset(qbl)
    reset(case)
    qbl.delete()
    case.delete()
    return success_bool, operand

Recall that GQSP is probabilistic, and the desired transformation is successfully applied if the auxiliary QuantumBool and QuantumFloat are both measured in state $\ket{0}$. 
Hence, the filtered state preparation is embedded within a [Repeat-Until-Success](../../reference/Jasp/Control%20Flow/RUS.rst) protocol.

In [ ]:
@jaspify(terminal_sampling=True)
def main(): 

    E = H.expectation_value(filtered_psi_prep, precision=0.001)()
    return E

#print(main())
# -0.6187347347347347 

And voila, we again achieve a significantly lower energy -- hence, a higher overlap with the ground state -- for the transformed state after QSP. 